In [40]:
# importing function from a different ipny 
from ipynb.fs.full.dataset import spam_test_train_set
from ipynb.fs.full.dataset import confusion
import pandas as pd
import collections
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")

In [17]:
[train_email, test_email] = spam_test_train_set()

2018-12-02 19:53:05,213   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207.12.141.100 by coliform.funmail.co.uk InterMail vK.4.04.00.00 658-589-341 license 893694jz34ml9wal1qv276h1o63n3u76 with ESMTP id <20034613789721.SXCH195.coliform@anderson.doramail.com> for <kruscit@enron.com>; Sun, 5 Aug 2001 20:34:59 -0600 Received: from Matthew 206.81.96.2 by anderson.doramail.com iPlanet Messaging Server 5.2 HotFix 1.14 built Mar 18 2003 with ESMTPA id <0OMZ008USAR04O@anderson.doramail.com> for kruscit@enron.com; Sun, 5 Aug 2001 23:34:59 -0300 IDT
2018-12-02 19:53:05,516   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207

In [81]:
def iter_spacy(strs, tokenizer):
    for s in strs:
        tokens = tokenizer(s)
        counts = collections.Counter(tokens)
        yield counts

In [82]:
bows_spacy_train = pd.DataFrame(iter_spacy(train_email.subject, tokenizer )).fillna(0).astype(int)
y_train= train_email['isspam']
bows_spacy_test = pd.DataFrame(iter_spacy(test_email.subject, tokenizer )).fillna(0).astype(int)
y_test= test_email['isspam']

In [83]:
bows_spacy_test.head()

,Change,Infrastructure,Schedule,Schedule,The,Schedule,GPCM,Employee,Energy,1-18-02.xls,...,to,Effective,Drop,trad,,e,date,June,"21,",2001
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
bows_spacy_train.head()

,CP&L,IM,News,8/3/01,TRRC,FW:,RE:,News,RE:,Capacity,...,Frontier,FUTURES,meeting:,Gas,AND,is,an,OPTIONS,URGENT,REQUEST
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
C = [0.01, 0.1, 1, 10, 100]
accuracy =[]
prec = []
true_pos =[]
false_pos = []
false_neg =[]
true_neg = []
for i in range(5):
    logreg = LogisticRegression(C=C[i])

    # Fit the classifier to the training data
    logreg.fit(bows_spacy_train, y_train)

    # Predict the labels of the test set: y_pred
    #y_pred =logreg.predict(bows_spacy_test)
    y_pred_train =logreg.predict(bows_spacy_train)
    true_pos.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[0])
    false_pos.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[1])
    false_neg.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[2])
    true_neg.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[3])
    accuracy.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[4])
    prec.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[5])

/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6

### Accuracy and Precision on training set

In [95]:
metrics = pd.DataFrame(columns = ['true_pos','false_pos','false_neg','true_neg','Accuracy', 'Precision'],\
                      index = ['C=0.01', 'C=0.1', 'C=1' ,'C=10','C=100'])

In [96]:
metrics.Accuracy = accuracy
metrics.Precision = prec
metrics.true_pos = true_pos
metrics.true_neg = true_neg
metrics.false_neg = false_neg
metrics.false_pos = false_pos

In [97]:
metrics.head()

,true_pos,false_pos,false_neg,true_neg,Accuracy,Precision
C=0.01,800.0,11.0,0.0,789.0,0.993125,1.0
C=0.1,800.0,11.0,0.0,789.0,0.993125,1.0
C=1,800.0,11.0,0.0,789.0,0.993125,1.0
C=10,800.0,11.0,0.0,789.0,0.993125,1.0
C=100,800.0,11.0,0.0,789.0,0.993125,1.0
